In [1]:
import serial
import serial.tools.list_ports

import threading
import time

from IPython.display import display

import numpy as np

In [2]:
class STmic:
    def __init__(self):
        ports=serial.tools.list_ports.comports()
        for p in ports:
            if p.vid==61525 and p.pid==38912:
                self.device=serial.Serial(p.device,baudrate=115200)
        if not hasattr(self,'device'):
            raise Exception('No controller unit detected')
            
    def __del__(self):
        self.device.close()
        
    
    #this method reads voltage for 2v scale
    def read_voltage(self):
        cmd = "m1"+"200000"+ "1" + "160" + "130" + "1" + "160" + "130"+ "\r"
        bytedata=bytearray(4*4)
        self.device.reset_output_buffer()
        self.device.reset_input_buffer()
        self.device.write(bytes(cmd,'utf-8'))
        self.device.readline()
        self.device.readinto(bytedata)
        data=np.frombuffer(bytedata,dtype='uint16').reshape((2,4))
        raw1=7.9000*(1.94-1.5*data[0,:]/1700)
        return raw1
    
    def read_voltage_avg(self):
        return float(np.array(self.read_voltage(),dtype=np.float64).mean())

In [3]:

#motor related variables
HIGH_VOLTAGE_THRESHOLD = 3.0 #CONST #threshold before the voltage becomes high from undefined
LOW_VOLTAGE_THRESHOLD = 0.5 #CONST #theshold before the voltage becomes undefined
previous_voltage_state = 0 #0 for low voltage 1 for high voltage
motor_phase_enum = 0 #0-3199 for each hall sensor
#projector related variables
current_image = 0 #current image index
projection_state = 0 #1 for start, 0 for stop, 2 for starting
image_lengths = [] #image_lengths = [1,3,3] means image 0 exposure duration is 1, image 1 exposure duration is 3,image 3 exposure duration is 3
exposure_counts = 0 #number of exposures the current image has already experienced



#function digitises voltages level, outputs -1 for undefined voltage levels
#https://cdn.phidgets.com/docs/images/thumb/0/00/LogicLevel_visualization.jpg/450px-LogicLevel_visualization.jpg
def digitise_voltage(voltage):
    if voltage >= HIGH_VOLTAGE_THRESHOLD:
        return 1
    if voltage <= LOW_VOLTAGE_THRESHOLD:
        return 0
    else:
        return -1
    
#also updates the previous voltage state when called if changed
def is_voltage_changed(controller):
    detected_voltage = controller.read_voltage_avg()
    if (detected_voltage > HIGH_VOLTAGE_THRESHOLD and previous_voltage_state == 0) or (detected_voltage < LOW_VOLTAGE_THRESHOLD and previous_voltage_state == 1):
        previous_voltage_state = 1 - previous_voltage_state
        return 1
    else:
        return 0

def send_trigger():
    pass#TODO

def reset_projector():
    projection_state = 0
    exposure_counts = 0
    current_image = 0

def reset_motor():
    previous_voltage_state = 0
    motor_phase_enum = 0
    pass

#this function is called when projector sends "image ready signal"
def initiate_pulses():
    motor_state = 2

def output_pulse():
    print("pulse outputted")
    
def start_operation(controller):
    while projection_state != 0:
        if projection_state == 0:
            reset_projector()
            break
        if projection_state == 2 and motor_phase_enum == 1:
            projection_state == 1
        if is_voltage_changed(controller):
            motor_phase_enum += 1
            if projection_state == 1:
                exposure_counts += 1
                send_trigger()#TODO
            if exposure_counts == image_lengths[current_image]:
                current_image += 1
                exposure_counts = 0
    del controller
    print("operation ended")
    return



In [25]:
controller = STmic()
print(controller.read_voltage_avg())
del controller

0.21027941176470544
